In [1]:
import torch
import torch.nn as nn

In [4]:
nn.LSTM(10,512)

OrderedDict([('weight_ih_l0',
              tensor([[ 0.0347, -0.0121,  0.0371,  ...,  0.0350,  0.0330, -0.0410],
                      [-0.0036,  0.0377,  0.0113,  ..., -0.0063,  0.0230, -0.0154],
                      [-0.0399,  0.0184, -0.0208,  ..., -0.0327,  0.0436, -0.0063],
                      ...,
                      [-0.0089,  0.0012,  0.0325,  ...,  0.0225, -0.0401,  0.0268],
                      [ 0.0249, -0.0166,  0.0325,  ..., -0.0277,  0.0402,  0.0343],
                      [-0.0092, -0.0181,  0.0150,  ..., -0.0039, -0.0364,  0.0251]])),
             ('weight_hh_l0',
              tensor([[-0.0030, -0.0377, -0.0078,  ...,  0.0413,  0.0440, -0.0041],
                      [-0.0007, -0.0378, -0.0372,  ...,  0.0415,  0.0037, -0.0153],
                      [-0.0379, -0.0074,  0.0315,  ..., -0.0229, -0.0222,  0.0081],
                      ...,
                      [ 0.0215,  0.0225,  0.0276,  ...,  0.0018,  0.0141, -0.0224],
                      [-0.0107,  0.0433, -0